# BIDS Bro Inspection

In [4]:
import glob, os
import pandas as pd
import pdb
from IPython.core import display as ICD



*make script to automize gathering of subject count, etc...*  
### DICOM directory:
**The directory-**  
`$ pwd`  
`/projects/niblab/bids_projects/Experiments/bro/bids/sourcedata/DICOM`     
`$ ls -d ses-1/sub-* | wc -l  50`    
`$ ls -d ses-2/sub-* | wc -l  51`  
### BIDS directory:  
`$ pwd`  
`/projects/niblab/bids_projects/Experiments/bro/bids`  
<br>
`$ ls -d ses-1/sub-* | wc -l  43`  
`Subjects missing from BIDS directory: 	{'sub-054', 'sub-013', 'sub-017', 'sub-035', 'sub-003', 'sub-004', 'sub-018'} `  
<br>
`$ ls -d ses-2/sub-* | wc -l  50`  
`Subjects missing from BIDS directory: 	{'sub-055'} `  

### Inspecting Scan Notes  
### Idea: pull out notes for subjects with questionable runs 

In [5]:
scan_notes_df=pd.read_csv("/Users/nikkibytes/Documents/niblunc/BRO/scan_notes_10_2019.34.csv")

In [6]:
scan_notes_df.columns.values

array(['Recorded Date', 'Subject ID (bro_XXX)',
       'Which testing visit is this?', 'Scout',
       'Scout notes series number', 'MP-RAGE',
       'MP-RAGE notes series number', 'Fieldmap',
       'Fieldmap notes series number', 'Training task run01',
       'Training 01 Series Number and notes', 'Training task run02',
       'Training 02 Series Number and notes', 'Resting state',
       'Resting State Series Number and notes',
       'Prediction error task run01',
       'Prediction Error 01 Series Number and notes',
       'Prediction error task run02',
       'Prediction Error 02 Series Number and notes',
       'General Notes on the scan:'], dtype=object)

In [63]:
scan_notes_df.sort_values('Subject ID (bro_XXX)', inplace=True)
#scan_notes_df.head()

In [8]:
sessions=['ses-1', 'ses-2']

## Checking the Volume 
Script to produce volumes file(run on renci): [`bids_quality_check.py`](https://github.com/niblunc/BRO/blob/master/BIDS/qa/bids_quality_check.py)   
*add functionality here to produce text file automatically*   
See [`CHANGELOG`](https://github.com/niblunc/BRO/blob/master/CHANGELOG.md) for further details on volume check and corrections status.


In [60]:
filename='/Users/nikkibytes/Documents/niblunc/bro/bids/qa/bad_volumes.txt'
vol_dict = {}
with open(filename) as f:
    content=f.readlines()
    for line in content:
        if "training" in line:
            file=line.split("\t")[0]
            vol=line.split("\t")[1].strip()
        else:
            file=line.split("\t\t")[0]
            vol=line.split("\t\t")[1].strip()
        temp_tuple=(file,vol)
        sub_id = file.split(":")[1].split("_")[0]
        if sub_id not in vol_dict:
            vol_dict[sub_id] = [temp_tuple]
        else:
            vol_dict[sub_id].append(temp_tuple)



In [78]:
print("Here we look at files found not matching their expected volume. \nExpectations: { pe: 193, training: 243, resting: 147 }")
  
for sub in vol_dict:
    # get information from resources to print specific subject information
    bro_id = sub.replace("sub-", "bro_")
    train=[x for x in vol_dict[sub] if "training" in x[0]]
    pe=[x for x in vol_dict[sub] if "pe" in x[0]]
    rest=[x for x in vol_dict[sub] if "rest" in x[0]]
    bro_df=scan_notes_df.loc[scan_notes_df['Subject ID (bro_XXX)'] == bro_id]
    print("\n-----------------------------------------------")
    print("\nID: {}\n".format(sub))
    print("> BAD VOLUME FILES")
    if train:
        print("\n- Training files: \n")
        for file, vol in train:
            print("{} \t{}".format(file,vol))
        lst=['Which testing visit is this?', 'Training task run01',
       'Training 01 Series Number and notes', 'Training task run02',
       'Training 02 Series Number and notes', 'General Notes on the scan:']
        ICD.display(bro_df[lst].sort_values("Which testing visit is this?"))
        print("\n")
    if pe:
        print("\n- Prediction Error files: \n")
        for file, vol in pe:
            print("{} \t\t{}".format(file,vol))
        lst=['Which testing visit is this?', 'Prediction error task run01',
       'Prediction Error 01 Series Number and notes',
       'Prediction error task run02',
       'Prediction Error 02 Series Number and notes', 'General Notes on the scan:']
        ICD.display(bro_df[lst].sort_values("Which testing visit is this?"))
        print("\n")
    if rest:
        print("\n- Resting files: \n")
        for file, vol in rest:
            print("{} \t\t{}".format(file,vol))
        lst=['Which testing visit is this?', 'Resting State Series Number and notes', 'General Notes on the scan:']
        ICD.display(bro_df[lst].sort_values("Which testing visit is this?"))
        print("\n")
            
    print("\n> SCAN NOTES")
    ICD.display(bro_df.sort_values("Which testing visit is this?"))
    print("\n\n")

Here we look at files found not matching their expected volume. 
Expectations: { pe: 193, training: 243, resting: 147 }

-----------------------------------------------

ID: sub-006

> BAD VOLUME FILES

- Training files: 

File:sub-006_ses-1_task-training_run-1_bold.nii.gz  	Volume:182
File:sub-006_ses-1_task-training_run-2_bold.nii.gz  	Volume:185


,Which testing visit is this?,Training task run01,Training 01 Series Number and notes,Training task run02,Training 02 Series Number and notes,General Notes on the scan:
21,Visit 1,Complete,Good,Complete,Seemed a little more hesitant in starting,NaN
22,Visit 2,Complete,Good,Complete,Good,NaN





- Prediction Error files: 

File:sub-006_ses-1_task-pe_run-1_bold.nii.gz  		Volume:141
File:sub-006_ses-1_task-pe_run-2_bold.nii.gz  		Volume:141


,Which testing visit is this?,Prediction error task run01,Prediction Error 01 Series Number and notes,Prediction error task run02,Prediction Error 02 Series Number and notes,General Notes on the scan:
21,Visit 1,Complete,Aight,Complete,Spiffy,NaN
22,Visit 2,Complete,Good,Complete,Good,NaN





- Resting files: 

File:sub-006_ses-1_task-resting_bold.nii.gz  		Volume:93


...

,Which testing visit is this?,Training task run01,Training 01 Series Number and notes,Training task run02,Training 02 Series Number and notes,General Notes on the scan:
100,Visit 1,Complete,Good,Complete,Good,Really good job throughout just had to restart...
105,Visit 2,Complete,Good,Complete,Good,Great job throughout





- Prediction Error files: 

File:sub-055_ses-1_task-pe_run-1_bold.nii.gz  		Volume:190
File:sub-055_ses-1_task-pe_run-2_bold.nii.gz  		Volume:192


,Which testing visit is this?,Prediction error task run01,Prediction Error 01 Series Number and notes,Prediction error task run02,Prediction Error 02 Series Number and notes,General Notes on the scan:
100,Visit 1,Complete,"Pumps stopped responding 2ish min in, restart...",Complete,Good,Really good job throughout just had to restart...
105,Visit 2,Complete,Good,Complete,Good,Great job throughout





> SCAN NOTES


,Recorded Date,Subject ID (bro_XXX),Which testing visit is this?,Scout,Scout notes series number,MP-RAGE,MP-RAGE notes series number,Fieldmap,Fieldmap notes series number,Training task run01,Training 01 Series Number and notes,Training task run02,Training 02 Series Number and notes,Resting state,Resting State Series Number and notes,Prediction error task run01,Prediction Error 01 Series Number and notes,Prediction error task run02,Prediction Error 02 Series Number and notes,General Notes on the scan:
100,8/20/19 17:25,bro_055,Visit 1,Complete,Good,Complete,Good,Complete,"Really big brain, had to trim parietal and ski...",Complete,Good,Complete,Good,Complete,Good,Complete,"Pumps stopped responding 2ish min in, restart...",Complete,Good,Really good job throughout just had to restart...
105,9/26/19 16:15,bro_055,Visit 2,Complete,Good,Complete,Good,Complete,Good,Complete,Good,Complete,Good,Complete,Good,Complete,Good,Complete,Good,Great job throughout


## Inspecting BIDS folders  
We need to check we have our expected files for the BIDS directory.   
* Input: previously generated `.csv` files containing a dataframe of file counts for subject directories in BIDS. [*link to code will be placed here*]()

In [113]:

for sess_id in sessions:
    
    
    if "1" in sess_id:
        print("\n> SESSION 1\n")s
        visit="Visit 1"
    else:
        print("\n> SESSION 2\n")
        visit="Visit 2"
    for csv in ['bro_{}_missing.csv'.format(sess_id), 'bro_{}_partial_missing.csv'.format(sess_id),'bro_{}_found.csv'.format(sess_id) ]:
        ses_df=pd.read_csv("csvs/"+csv ,sep="\t")
        ses_df.set_index("Unnamed: 0", inplace=True)
        ses_df.index.names=["subjects"]
        if ("missing" or "partial") in csv:
    
            for sub in ses_df.index.values:
                print("> {} \n".format(sub))
                bro_id = sub.replace("sub-", "bro_")
                ICD.display(ses_df.loc[[sub]])
                bro_df=scan_notes_df.loc[scan_notes_df['Subject ID (bro_XXX)'] == bro_id]
                print("\n")
                ICD.display(bro_df[['General Notes on the scan:', 'Which testing visit is this?']].loc[bro_df['Which testing visit is this?']==visit])
                #ICD.display(bro_df.sort_values("Which testing visit is this?"))
                print("\n")

    
    


> SESSION 1

> sub-001 



,anat,fmap,func,runs_pe,runs_rest,runs_train
subjects,,,,,,
sub-001,0,0,0,0,0,0


,General Notes on the scan:,Which testing visit is this?
1,Really good,Visit 1




> sub-002 



,anat,fmap,func,runs_pe,runs_rest,runs_train
subjects,,,,,,
sub-002,0,0,0,0,0,0


...

,anat,fmap,func,runs_pe,runs_rest,runs_train
subjects,,,,,,
sub-025,1,0,0,0,0,0


,General Notes on the scan:,Which testing visit is this?
45,"Good job staying still, just had motion from s...",Visit 2




> sub-042 



,anat,fmap,func,runs_pe,runs_rest,runs_train
subjects,,,,,,
sub-042,0,3,3,2,1,0


,General Notes on the scan:,Which testing visit is this?


## Reference and Code


### run this on renci to get output for sessions count 

In [58]:
""" 


ses1_dcms=[x.split("/")[-1] for x in glob.glob('/projects/niblab/bids_projects/Experiments/bro/bids/sourcedata/DICOM/ses-1/sub-*')]
ses2_dcms=[x.split("/")[-1] for x in glob.glob('/projects/niblab/bids_projects/Experiments/bro/bids/sourcedata/DICOM/ses-2/sub-*')]

ses1_dcms_ct= len(ses1_dcms)
ses2_dcms_ct= len(ses2_dcms)

ses1_bids=[x.split("/")[-2] for x in glob.glob('/projects/niblab/bids_projects/Experiments/bro/bids/sub-*/ses-1')]
ses2_bids=[x.split("/")[-2] for x in glob.glob('/projects/niblab/bids_projects/Experiments/bro/bids/sub-*/ses-2')]

ses1_bids_ct= len(ses1_bids)
ses2_bids_ct= len(ses2_bids)


ses1_mia_lst=set(ses1_dcms) - set(ses1_bids)
ses2_mia_lst=set(ses2_dcms) - set(ses2_bids)

if not ses1_mia_lst:
    ses1_mia_lst="None missing"
if not ses2_mia_lst:
    ses2_mia_lst="None missing"
    
print("> Session 1: \nDICOM subject directory count: \t{} \
      \nBIDS subject directory count: \t{} \nSubjects missing from BIDS directory: \t{} \n \
      \n\n> Session 2: \nDICOM subject directory count: \t{} \
      \nBIDS subject directory count: \t{} \
      \nSubjects missing from BIDS directory: \t{} \n".format(ses1_dcms_ct,ses1_bids_ct,ses1_mia_lst,ses2_dcms_ct,ses2_bids_ct, ses2_mia_lst))"""

' run this on renci to get output\n\n\nses1_dcms=[x.split("/")[-1] for x in glob.glob(\'/projects/niblab/bids_projects/Experiments/bro/bids/sourcedata/DICOM/ses-1/sub-*\')]\nses2_dcms=[x.split("/")[-1] for x in glob.glob(\'/projects/niblab/bids_projects/Experiments/bro/bids/sourcedata/DICOM/ses-2/sub-*\')]\n\nses1_dcms_ct= len(dcms_ses1)\nses2_dcms_ct= len(dcms_ses2)\n\nses1_bids=[x.split("/")[-2] for x in glob.glob(\'/projects/niblab/bids_projects/Experiments/bro/bids/sub-*/ses-1\')]\nses2_bids=[x.split("/")[-2] for x in glob.glob(\'/projects/niblab/bids_projects/Experiments/bro/bids/sub-*/ses-2\')]\n\nses1_bids_ct= len(ses1_bids)\nses2_bids_ct= len(ses2_bids)\n\n\nses1_mia_lst=set(ses1_dcms) - set(ses1_bids)\nses2_mia_lst=set(ses2_dcms) - set(ses2_bids)\n\nif not ses1_mia_lst:\n    ses1_mia_lst="None missing"\nif not ses2_mia_lst:\n    ses2_mia_lst="None missing"\n    \nprint("> Session 1: \nDICOM subject directory count: \t{}       \nBIDS subject directory count: \t{} \nSubjects mis